In [ ]:
#| default_exp api

In [ ]:
#| export
from datetime import datetime
from typing import List, Union, Dict
from pino_inferior.message import Message
from pino_inferior.models import aengine, APICommentQuery, APICommentQueryTaskMapping, APITask, \
    APIContextSummarizationQuery, APIContextSummarizationTaskMapping
from sqlalchemy.ext.asyncio import AsyncEngine, AsyncSession
from sqlalchemy import select
import pandas as pd
from pydantic.dataclasses import dataclass
from fastapi import FastAPI, HTTPException
from subprocess import Popen, STDOUT, PIPE
from threading import Thread
from time import sleep
from pydantic import TypeAdapter
import requests
from pino_inferior.context_extractor import build_context_extractor_chain, \
    PromptMarkupConfig as ContextExtractorPromptMarkupConfig, \
    LengthConfig as ContextExtractorPromptLengthConfig, \
    CONTEXT_INPUT_TEXT, CONTEXT_INPUT_POST_TIME, CONTEXT_INPUT_GOALS, \
    CONTEXT_INPUT_CURRENT_TIME, CONTEXT_INPUT_USERNAME, CONTEXT_INPUT_CHARACTER, \
    CONTEXT_OUTPUT_CONTEXT
from langchain.chat_models import ChatOpenAI
from pino_inferior.core import OPENAI_API_KEY, OPENAI_AGENT_MODEL, OPENAI_CONTEXT_MODEL, OPENAI_FALLACY_MODEL, \
    VECTOR_DB, VECTOR_DB_PARAMS, MEMORY_PARAMS
from pino_inferior.memory import Memory, INPUT_RETRIEVER_QUERY, OUTPUT_RETRIEVER_DOCUMENTS
import tiktoken
from traceback import format_exc
from pino_inferior.agent import TOOLS_PROMPTS_DIR, ToolDescription, RolePlayAgent, \
    PromptMarkupConfig as RPPromptMarkupConfig, \
    LengthConfig as RPLengthMarkupConfig, \
    AGENT_INPUT_TIME, AGENT_INPUT_CONTEXT, AGENT_INPUT_FALLACIES, AGENT_INPUT_HISTORY, AGENT_INPUT_TOOLS, \
    AGENT_INPUT_USERNAME, AGENT_INPUT_CHARACTER, AGENT_INPUT_GOAL, AGENT_INPUT_STYLE_EXAMPLES, \
    AGENT_INPUT_STYLE_DESCRIPTION
from pino_inferior.fallacy import build_fallacy_detection_chain, LengthConfig as FallacyLengthConfig, \
    read_fallacies, FALLACIES_FNAME, \
    INPUT_QUERY as INPUT_FALLACY_QUERY, OUTPUT_SHORT_ANSWER as OUTPUT_FALLACY_QUERY
import os
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
#| export
TASK_STATUS_NOT_STARTED = "not_started"
TASK_STATUS_IN_PROGRESS = "in_progress"
TASK_STATUS_FINISHED = "finished"
TASK_STATUS_FAILED = "failed"

In [ ]:
#| export
app = FastAPI()

In [ ]:
#| export
@dataclass
class User:
    name: str
    character: str
    goals: str


@dataclass
class UserWithStyle(User):
    style_example: str
    style_description: str


@dataclass
class MessageQuery:
    author: str
    time: str
    content: str


@dataclass
class TaskResponse:
    id: int
    status: str
    response: str


@dataclass
class CommentRequest:
    context: str
    history: List[MessageQuery]
    user: UserWithStyle

## API requests

### Thread commenting task creation query

In [ ]:
#| export
async def _comment_query(engine: AsyncEngine,
                         time: datetime,
                         context: str,
                         history: List[Message],
                         user_name: str,
                         user_character: str,
                         user_goals: str,
                         user_style_example: str,
                         user_style_description: str) -> TaskResponse:
    async with AsyncSession(engine) as session:
        async with session.begin():
            query = APICommentQuery(
                time=time,
                context=context,
                history=[
                    message.to_dict()
                    for message in history
                ],
                user_name=user_name,
                user_character=user_character,
                user_goals=user_goals,
                user_style_example=user_style_example,
                user_style_description=user_style_description,
            )
            session.add(query)
            task = APITask(
                status=TASK_STATUS_NOT_STARTED,
                response="",
                created_at=time,
                updated_at=time,
            )
            session.add(task)
            await session.flush()
            await session.refresh(query)
            await session.refresh(task)
            mapping = APICommentQueryTaskMapping(
                comment_id=query.query_id,
                task_id=task.task_id,
            )
            session.add(mapping)
            return TaskResponse(task.task_id, task.status, task.response)

In [ ]:
#| export
@app.post("/comment")
async def api_comment_query(request: CommentRequest) -> TaskResponse:
    return await _comment_query(
        aengine,
        time=datetime.now(),
        context=request.context,
        history=[Message.from_dict(item.__dict__) for item in request.history],
        user_name=request.user.name,
        user_character=request.user.character,
        user_goals=request.user.goals,
        user_style_example=request.user.style_example,
        user_style_description=request.user.style_description,
    )

### Context extraction task creation query

In [ ]:
#| export
async def _context_query(engine: AsyncEngine,
                         time: datetime,
                         post_time: datetime,
                         text: str,
                         user_name: str,
                         user_character: str,
                         user_goals: str) -> TaskResponse:
    async with AsyncSession(engine) as session:
        async with session.begin():
            query = APIContextSummarizationQuery(
                text=text,
                time=time,
                post_time=post_time,
                user_name=user_name,
                user_character=user_character,
                user_goals=user_goals,
            )
            session.add(query)
            task = APITask(
                status=TASK_STATUS_NOT_STARTED,
                response="",
                created_at=time,
                updated_at=time,
            )
            session.add(task)
            await session.flush()
            await session.refresh(task)
            await session.refresh(query)
            mapping = APIContextSummarizationTaskMapping(
                context_id=query.query_id,
                task_id=task.task_id,
            )
            session.add(mapping)
            return TaskResponse(task.task_id, task.status, task.response)

In [ ]:
#| export
@dataclass
class Post:
    text: str
    time: str


@dataclass
class ContextRequest:
    post: Post
    user: User

In [ ]:
#| export
@app.post("/context")
async def api_context_query(request: ContextRequest) -> TaskResponse:
    return await _context_query(
        aengine,
        datetime.now(),
        pd.to_datetime(request.post.time),
        request.post.text,
        request.user.name,
        request.user.character,
        request.user.goals,
    )

### Status retrieving query

In [ ]:
#| export
@app.get("/task-status")
async def api_task_status(task_id: int) -> TaskResponse:
    async with AsyncSession(aengine) as session:
        async with session.begin():
            query = select(APITask).filter(APITask.task_id == task_id)
            query_result = await session.execute(query)
            task = query_result.scalar()
            if not task:
                raise HTTPException(404)
            return TaskResponse(
                task.task_id,
                task.status,
                task.response
            )

### Get tasks for execution

In [ ]:
#| export
@dataclass
class ContextRequestWithTime(ContextRequest):
    time: str


@dataclass
class CommentRequestWithTime(CommentRequest):
    time: str


@dataclass
class TaskForExecution:
    task_id: int
    task_type: str
    task_objects: Dict[str, Union[ContextRequestWithTime, CommentRequestWithTime]]

In [ ]:
#| export
@app.get("/tasks-for-execution")
async def api_tasks_for_execution(count: int) -> List[TaskForExecution]:
    async with AsyncSession(aengine) as session:
        async with session.begin():
            tasks_to_mark_query = select(APITask) \
                .filter(APITask.status == TASK_STATUS_NOT_STARTED) \
                .order_by(APITask.created_at) \
                .limit(count)
            tasks_to_mark_response = await session.execute(tasks_to_mark_query)
            tasks_to_mark = tasks_to_mark_response.scalars().all()
            for task in tasks_to_mark:
                task.status = TASK_STATUS_IN_PROGRESS
            
            task_ids = {
                task.task_id
                for task in tasks_to_mark
            }
            await session.commit()
        async with session.begin():
            result_tasks = []
            comment_query = select(APICommentQuery, APICommentQueryTaskMapping.task_id) \
                .join(APICommentQueryTaskMapping, APICommentQueryTaskMapping.comment_id == APICommentQuery.query_id) \
                .filter(APICommentQueryTaskMapping.task_id.in_(task_ids))
            comment_query_response = await session.execute(comment_query)
            comment_request: APICommentQuery
            task_id: int
            for comment_request, task_id in comment_query_response.all():
                result_tasks.append(TaskForExecution(
                    task_id=task_id,
                    task_type="comment",
                    task_objects={
                        "comment": CommentRequestWithTime(
                            context=comment_request.context,
                            history=comment_request.history,
                            user=UserWithStyle(
                                name=comment_request.user_name,
                                character=comment_request.user_character,
                                goals=comment_request.user_goals,
                                style_example=comment_request.user_style_example,
                                style_description=comment_request.user_style_description,
                            ),
                            time=str(comment_request.time),
                        )
                    }
                ))
            context_query = select(APIContextSummarizationQuery, APIContextSummarizationTaskMapping.task_id) \
                .join(APIContextSummarizationTaskMapping, APIContextSummarizationTaskMapping.context_id == APIContextSummarizationQuery.query_id) \
                .filter(APIContextSummarizationTaskMapping.task_id.in_(task_ids))
            context_query_response = await session.execute(context_query)
            context_request: APIContextSummarizationQuery
            task_id: int
            for context_request, task_id in context_query_response.all():
                result_tasks.append(TaskForExecution(
                    task_id=task_id,
                    task_type="context",
                    task_objects={
                        "context": ContextRequestWithTime(
                            post=Post(
                                text=context_request.text,
                                time=str(context_request.post_time),
                            ),
                            user=User(
                                name=context_request.user_name,
                                character=context_request.user_character,
                                goals=context_request.user_goals,
                            ),
                            time=str(context_request.time),
                        )
                    }
                ))
            return result_tasks

### Set task statuses

In [ ]:
#| export
@app.post("/task-status-set")
async def task_status_set(request: List[TaskResponse]) -> None:
    async with AsyncSession(aengine) as session:
        async with session.begin():
            update_time = datetime.now()
            task_id2task = {
                task.id: task
                for task in request
            }
            tasks_to_mark_query = select(APITask) \
                .filter(APITask.task_id.in_(task_id2task.keys()))
            task_to_mark_query_response = await session.execute(tasks_to_mark_query)
            task: APITask
            for task, in task_to_mark_query_response.all():
                print("TASK CLASS", task.__class__)
                task.updated_at = update_time
                task.response = task_id2task[task.task_id].response
                task.status = task_id2task[task.task_id].status

### Execute tasks

In [ ]:
#| export
def _parse_task(data: dict) -> TaskForExecution:
    return TypeAdapter(TaskForExecution).validate_python(data)


def _parse_tasks(data: List[dict]) -> List[TaskForExecution]:
    return TypeAdapter(List[TaskForExecution]).validate_python(data)

In [ ]:
#| export
INTERMEDIATE_TASK_ID = "task_id"

In [ ]:
agent_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_AGENT_MODEL)
agent_encoding = tiktoken.encoding_for_model(agent_model.model_name)
fallacy_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_FALLACY_MODEL)
fallacy_encoding = tiktoken.encoding_for_model(fallacy_model.model_name)
context_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_CONTEXT_MODEL)
context_encoding = tiktoken.encoding_for_model(context_model.model_name)

In [ ]:
context_extractor = build_context_extractor_chain(
    context_model,
    lengths=ContextExtractorPromptLengthConfig(
        cut_function=lambda text, length: context_encoding.decode(context_encoding.encode(text)[:length]),
        length_function=lambda text: len(context_encoding.encode(text)),
        max_goals_length=512,
        max_name_length=10,
        max_character_length=512,
        max_post_length=2048,
    ),
    prompts=ContextExtractorPromptMarkupConfig(
        tags_open_sequence="[tags]",
        tags_close_sequence="[/tags]",
        summary_open_sequence="[summary]",
        summary_close_sequence="[/summary]",
    )
)

In [ ]:
def _read_file(fname: str) -> str:
    with open(fname, "r", encoding="utf-8") as src:
        return src.read()

In [ ]:
fallacy_length_config = FallacyLengthConfig(
    length_function=lambda text: len(fallacy_encoding.encode(text)),
    max_messages_length=2048,
    max_fallacies_length=4096,
)
fallacy_tool = (
    ToolDescription(
        name="fallacy",
        description=_read_file(os.path.join(TOOLS_PROMPTS_DIR, "fallacy.txt")),
        input_key=INPUT_FALLACY_QUERY,
        output_key=OUTPUT_FALLACY_QUERY,
    ),
    build_fallacy_detection_chain(fallacy_model, fallacy_length_config),
)

In [ ]:
sentence_encoder = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model="text-embedding-ada-002",
)
memory_container = Memory(
    engine=aengine,
    vector_db=VECTOR_DB(
        embedding_function=sentence_encoder,
        **VECTOR_DB_PARAMS
    ),
    **MEMORY_PARAMS
)
memory_tool = (
    ToolDescription(
        name="memory",
        description=_read_file(os.path.join(TOOLS_PROMPTS_DIR, "memory.txt")),
        input_key=INPUT_RETRIEVER_QUERY,
        output_key=OUTPUT_RETRIEVER_DOCUMENTS,
    ),
    memory_container.build_retriever_chain()
)
memory_tool

(ToolDescription(name='memory', description='External memory tool. \nYour training data is limited, but you can use it.\nQuery should be exact since memory will only see it, not the surrounding context.\nSometimes memory might return non-relevant results (like sudden associations outside the discussed context).\nReturns retrieved documents.\nCall it this way: [memory]%Your query[/memory][call]', input_key='query', output_key='documents_text'),
 TransformChain(input_variables=['query'], output_variables=['documents'], transform_cb=<function Memory.build_retriever_chain.<locals>._retrieve_documents>, atransform_cb=<function Memory.build_retriever_chain.<locals>._aretrieve_documents>)
 | TransformChain(input_variables=['documents'], output_variables=['documents_text'], transform_cb=<function Memory.build_retriever_chain.<locals>._stringify_documents>, atransform_cb=<function Memory.build_retriever_chain.<locals>._astringify_documents>))

In [ ]:
agent = RolePlayAgent(
    tools=[memory_tool, fallacy_tool],
    lengths=RPLengthMarkupConfig(
        cut_function=lambda text, length: agent_encoding.decode(agent_encoding.encode(text)[:length]),
        length_function=lambda text: len(agent_encoding.encode(text)),
    ),
    prompt_markup=RPPromptMarkupConfig(),
    llm=agent_model,
    max_iter=5,
)

In [ ]:
async def _execute_context_extractor_task(task: TaskForExecution) -> str:
    assert task.task_type == "context"
    request: ContextRequestWithTime = task.task_objects[task.task_type]
    inputs = {
        INTERMEDIATE_TASK_ID: task.task_id,
        CONTEXT_INPUT_TEXT: request.post.text,
        CONTEXT_INPUT_POST_TIME: pd.to_datetime(request.post.time),
        CONTEXT_INPUT_GOALS: request.user.goals,
        CONTEXT_INPUT_CURRENT_TIME: pd.to_datetime(request.time),
        CONTEXT_INPUT_USERNAME: request.user.name,
        CONTEXT_INPUT_CHARACTER: request.user.character
    }
    response = await context_extractor.ainvoke(inputs)
    return response[CONTEXT_OUTPUT_CONTEXT]

In [ ]:
async def _execute_comment_task(task: TaskForExecution) -> str:
    assert task.task_type == "comment"
    request: CommentRequestWithTime = task.task_objects[task.task_type]
    inputs = {
        AGENT_INPUT_TIME: pd.to_datetime(request.time),
        AGENT_INPUT_CONTEXT: request.context,
        AGENT_INPUT_FALLACIES: read_fallacies(FALLACIES_FNAME),
        AGENT_INPUT_HISTORY: [
            Message(message.author, pd.to_datetime(message.time), message.content)
            for message in request.history
        ],
        AGENT_INPUT_TOOLS: [
            fallacy_tool,
            memory_tool,
        ],
        AGENT_INPUT_USERNAME: request.user.name,
        AGENT_INPUT_CHARACTER: request.user.character,
        AGENT_INPUT_GOAL: request.user.goals,
        AGENT_INPUT_STYLE_EXAMPLES: request.user.style_example.split("\n\n"),
        AGENT_INPUT_STYLE_DESCRIPTION: request.user.style_description
    }
    return await agent.arun(inputs)

In [ ]:
async def _execute_task(task: TaskForExecution) -> str:
    types = {"context": _execute_context_extractor_task,
             "comment": _execute_comment_task}
    assert task.task_type in types
    return await types[task.task_type](task)

In [ ]:
async def _process_task(task: TaskForExecution) -> TaskResponse:
    try:
        response = await _execute_task(task)
        status = TASK_STATUS_FINISHED
    except:
        response = format_exc()
        status = TASK_STATUS_FAILED
    return TaskResponse(task.task_id, status, response)

In [ ]:
async def _process_tasks(tasks: List[TaskForExecution]) -> List[TaskResponse]:
    result = []
    for task in tasks:
        result.append(await _process_task(task))
    return result

## Examples

### Run server in background

In [ ]:
import nbdev; nbdev.nbdev_export()

In [ ]:
def _run_server(log_file: str) -> None:
    startup_complete = False

    def _thread():
        nonlocal startup_complete
        command = ["uvicorn", "pino_inferior.api:app", "--reload", "--host", "0.0.0.0", "--port", "8080"]
        process = Popen(command, stdout=PIPE, stderr=STDOUT)
        with open(log_file, "wb") as log:
            while process.poll() is None:
                line = process.stdout.readline()
                if b"Application startup complete." in line:
                    startup_complete = True
                log.write(line)
                log.flush()

    Thread(target=_thread).start()
    while not startup_complete:
        sleep(0.1)


_run_server("09_api_methods_server.log")

### Create context extraction task

In [ ]:
post_text = """Будет ли мобилизация осенью?⁠⁠
Самый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат. Хочу его разобрать и озвучить свои соображения на тему, будет ли мобилизация, если да – то когда.

Мой канал в ТГ: https://t.me/artjockey

Напомню старым и новым читателям, что у меня нет никаких инсайдов, я просто озвучу свое мнение и постараюсь его обосновать. С чем можно согласиться или не согласиться.

Будет ли мобилизация осенью? Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост
Первая волна мобилизации
Она началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать. То есть, формально, новую волну власть может не объявлять, а просто, опираясь на действующий указ, начать новый призыв.

Для чего нужна была мобилизация? Численность группировки вторжения ВС РФ была около 200 тысяч человек (с учетом войск еще независимых ДЛНР). Украинская армия мирного времени – примерно 260 тысяч человек. То есть, силы были примерно равны, даже в пользу ВС РФ, потому что у ВСУ это не вся сухопутная армия, а с учетом всех тыловых служб, штаба, пограничников и т.д.

Но сразу после начала войны, украинское правительство объявляет всеобщую мобилизацию, вполне логичный и правильный шаг. Численность украинской армии быстро увеличивается и к октябрю достигает 700 тысяч человек, по словам украинского министра обороны. Конечно, далеко не все эти люди находились на передовой, то есть, не являлись «штыками». Но перекос в численности воюющих армий уже очевиден и было понятно, что продолжать завоевательную войну имея в 2-3 раза меньше людей невозможно.

Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и продолжить войну в условно равном соотношении сил. Даже если перекос в чью-то сторону сохранялся, он был уже не в разы, а в десятки процентов, не более. Это позволило и равномерно укрепить фронт, чтобы не допустить повторения сценария наступления ВСУ на Изюм.

Но всеобщая мобилизация на территории Украины не прекратилась, украинская армия продолжила пополняться, зимой стали известны и планы Киева провести наступление в 2023-м году. В таких условиях уже было понятно, что вторая волна – неизбежна. ВС РФ необходимо, хотя бы, поддерживать паритет по численности. Многие источники писали, что мобилизация будет проведена в начале года, чтобы иметь обученные резервы к концу весны-лету. Я тоже думал, что мобилизацию объявят.

Стратегия Кремля
Как мы знаем, вторую волну мобилизации не объявили и связано это с курсом руководства России на изоляцию войны от основной массы российского общества. Я не сомневаюсь, что подготовка действительно велась, но в какой-то момент планы поменялись.

Для среднего россиянина война идет и, по этой стратегии, должна идти где-то в телевизоре, потому и не война, а СВО. Да, есть некий процент людей, кого затронула мобилизация в первую волну, но их, относительно общего населения страны, немного.

Также временно вопрос пополнения закрыл собой Вагнер, через который, как мы теперь знаем, прошло 78 тысяч человек, что довольно много. Это где-то 25% от частично мобилизованных.

Но общая проблема с поддержкой паритета по численности личного состава никуда не делась и ее нужно было как-то решать. Решение было озвучено весной, вместо второй волны мобилизации предполагается до конца года набрать 400 тысяч людей на контрактную службу. Вот эти новые контрактники и должны заменить собой «вторую волну».

Скрытая мобилизация
Есть такое понятие, как скрытая мобилизация. То есть, набор в войска не обязательно ведется с официальными объявлениями по телевизору, фанфарами и всесторонним освещением в СМИ.

При этом, не стоит понимать термин настолько буквально, что это некие тайные мероприятия, о которых никто не должен узнать. Совсем не обязательно. Это просто некий общий комплекс мер, призванный замаскировать или скрыть отдельные детали процесса или же прикрыть его юридически. Так, например, официальная мобилизация государства может являться поводом для объявления войны, а, скажем, увеличение численности пограничных войск и парамилитарных образований – таким поводом уже не будет.

Также, скрытая мобилизация может скрывать не сам процесс, а его детали. При частичной мобилизации все знали и сколько людей будет призвано, и куда они будут направлены. При скрытой мобилизации такие подробности могут не озвучиваться, хотя сам проходящий процесс и является достоянием общественности.

Вот нынешняя стратегия российского руководства и является проведением скрытой мобилизацией. Официально, контрактники набираются не для участия в СВО, а согласно стратегическому плану развития армии, которое не так давно было подписано Путиным и предусматривает увеличение численности ВС РФ и вне контекста войны.

Методы скрытой мобилизации
Сейчас в РФ задействовано три основных источника пополнения личного состава.

Во-первых, это прямой поток добровольцев, заключающих контракты на службу в рядах армии. Жители России, думаю, видели и агрессивную рекламу службы, кто-то сталкивался с агитаторами на работе, а кто-то даже получал повестку для уточнения данных в военкомат, где его уговаривали заключить контракт.

Во-вторых, это перевод на контракт срочников. Каждый срочник может заключить контракт, который пойдет в зачет срочки.

В-третьих, это продолжающийся набор в тюрьмах, в основном, в отряды Шторм вместо Вагнера.

Все эти три метода позволяют увеличить численность личного состава в рядах вооруженных сил, и они все задействованы. Но есть еще один метод, это уже мобилизованные, которым предлагают заключить контракт. Таки люди тоже идут в статистику, но вот на общее число солдат они не влияют.

Есть и менее очевидные способы, вот новость прям с Пикабу. Когда в ходе недавних массовых рейдов среди мест компактного проживания граждан, недавно получивших паспорт, выявляли тех, кто еще не стал на учет в военкомат и помогали им это сделать прям с доставкой к месту оформления документов.

Успехи скрытой мобилизации
Вот теперь о цифрах, как ни странно, они нам известны. Бывший президент, Дмитрий Медведев, который как раз и курирует этот процесс, озвучил, что с 1 апреля по август на контракт записалась 231 тысяча человек.

Это довольно много и почти догоняет общее число мобилизованных в первую волну. То есть, по факту, перекрывает потребности во второй волне мобилизации. И да, наверняка, люди сразу скажут о вот тех самых приписках, когда уже мобилизованных переводят на контракт, учитывают в статистике, но фактическое положение дел не меняется. К сожалению, мы не знаем, сколько таких людей среди этих 231 тысячи есть.

Но у нас есть взгляд и с другой стороны. Скрытую мобилизацию в РФ совсем недавно комментировал представитель ГУР Украины, он подтвердил, что «разными методами» в месяц мобилизуется 20 тысяч человек. То есть, даже с учетом всех манипуляций, поток новобранцев в зону СВО достаточно большой, в течение года это и будут те самые примерно 300 тысяч человек, которые были в первой волне.

Промежуточные выводы
Вторая волна мобилизации в России уже идет и идет достаточно успешно. Даже при мобилизации 20 тысяч человек в месяц, это в общем позволяет руководству РФ отказаться от проведения отдельной второй волны частичной мобилизации и закрыть все потребности за счет добровольцев (условно, все мы понимаем, что часть людей заключает контракт добровольно-принудительно).

Также вспоминаем, что я написал в начале – указ о частичной мобилизации юридически еще действует. Можно предположить, что при недостаточном потоке новобранцев будет проведена не вторая волна по образу первой, а еще включен еще один механизм. Начнут рассылать повестки ежемесячно в небольшом количестве, уже прямо призывая не 300 тысяч человек за раз, а, скажем, по 10 тысяч человек в месяц.

Именно так происходит на Украине. Процесс рассылки и раздачи повесток идет постоянно без каких-либо волн. Повестка – это вызов на «уточнение данных», где уже вместо нее вручают мобилизационное предписание, по украинской практике, даже не выпуская из военкомата. И напомню, что в России рассылаются похожие повестки и сейчас, только с целью уговорить заключить контракт, но при необходимости, просто начнут сразу давать предписания.

Юридические изменения
Между тем, Госдума ввела ряд изменений в законодательство, которые тоже серьезно всколыхнули общество. Во-первых, это увеличение возраста запаса, во-вторых, увеличение возраста призыва на срочную службу, в-третьих, штрафы для уклонистов.

Из этих изменений делается вывод, что мобилизация готовится осенью. Но это не совсем так, первые два пункта точно никак не могут повлиять на осеннюю мобилизацию. Возраст учета вообще вводится поэтапно, каждый год на один год. А возраст призыва увеличивается с 2024-го, но не с 2023-го года. На мой взгляд, это не подготовка к СВО вообще, а подготовка к вероятной эскалации конфликта. Я уже как-то говорил, что мы, возможно, находимся в 1938-м или в 1913-м году, просто этого еще не знаем. А лет через 20 будем спорить, является ли Российско-Украинская война отдельным конфликтом или это один из театров действий Третьей мировой.

Остается третий пункт, ужесточение штрафов для уклонистов. Кто-то это воспринял как легитимизацию взятки за уклонение, но ведь штраф не освобождает от обязанности явиться в военкомат по повестке.

Я считаю, что тут смысл в другом. Чуть выше я писал, что один из источников пополнения личного состава в зоне СВО – это срочники. И здесь тоже есть изменения, срочник сможет заключить контракт через месяц после призыва, а раньше мог только через три.

И вот эти все изменения направлены на то, чтобы упростить осенний призыв на срочную служба, а уже на срочке начнут обрабатывать призывников на тему подписания контракта. И здесь, конечно, к кому-то могут применить кнут, кто-то поведется на пряник в виде, в первую очередь, зарплаты. Штраф этому только поможет, ведь штрафник может записаться на контракт и с нескольких зарплат с ним расплатиться.

Есть и еще одно не самое заметное изменение. С недавнего времени, иностранцы могут заключить контракт с ВС РФ на один год, а затем получить гражданство по упрощенной процедуре.

Логические «против»
Кроме того, что реальные и юридические действия руководства России нацелены на то, чтобы избежать проведения второй волны частичной мобилизации, есть еще и объективные причины, почему ее не стоит проводить, во всяком случае, сейчас.

Проведений первой волны выявило ряд проблем, которые такой метод создает. И здесь даже можно обойтись без пруфов, просто само по себе очевидно, что одномоментный призыв в армию 300 тысяч человек за 1,5 месяца сильно перегружает все тыловые службы. Так, например, призыв на срочку – это 120-150 тысяч человек в течение 3-х месяцев.

При этом, на Украине, например, при всеобщей мобилизации отменили срочную службу, но в России при частичной – нет. И в прошлом году это привело к тому, что срок призыва на срочку пришлось сдвинуть на месяц.

Зима и поздняя осень – это неудобное время для подготовки личного состава. В отличие от срочников, которые, условно, будут «красить траву», эти люди отправляются на войну и нужно постараться их хоть как-то обучить. Зимой это не слишком удобно из-за погоды, температуры и всего остального. Если на полигоне грязь, в которой танк застрянет, обучения не получится.

Конечно, возникает правильный вопрос, если все эти проблемы очевидны даже с дивана, зачем тогда проводилась первая волна? Она была продиктована военной необходимостью, когда с ней очень долго тянули, а потом вынуждены были добирать людей на войну в условиях, что они нужны были еще на вчера (на начало сентября в Харьковской области).

Сейчас же ситуация на фронте явно не диктует срочной необходимости добирать людей. В моем прошлом посте я показал, что Россия проводит успешную оборонительную операцию, нет признаков обрушения фронта или возможных скорых успехов ВСУ. При том, у России хватает сил не только обороняться, но еще и наступать на другом фланге на Купянск.

Вопрос ротации
Еще один часто поднимаемый вопрос, это то, что мобилизация необходима для ротации, а то и для демобилизации частично мобилизованных. Потому что они уже почти год как воюют, а хотят по домам. Ну или хотя бы на ротацию.

Возможно, меня поправят действующие военные, но у меня сложилось мнение, что все эти нормативы по ротации, записанные в уставах, в нынешней войне пошли к одному месту с обеих сторон. В реальности, классических ротаций войск не проводит ни Россия, ни Украина, за исключением частей, понесших сильные потери. Для отдыха военнослужащим дают отпуска, я знаю лично мобилизованных, кто уже успел побывать дома, а кто-то и два раза каким-то образом, но само подразделение в тыл не отводится.

Что же касается демобилизации – ее не планируется точно и про это говорил Путин в июне на встрече с военкорами. Ему задавали этот вопрос, от ответа он ушел, сказав, что все будет зависеть от ситуации на фронте. Потому рассчитывать на демобилизацию через год я бы точно не стал. Возможно, такие процессы начнутся к концу второго года службы мобилизованных, но точно не в ближайшее время и Путин ясно дал это понять.

К тому же, среди мобилизованных активно идет агитация подписывать контракт, который на время СВО является бессрочным. Вот, возможно, как раз к концу второго года, когда уже и будут более точные расклады по ходу боевых действий с учетом вероятного наступления ВС РФ, начнут отпускать. Тогда получится, что кто-то выбыл из-за потерь, кто-то подписал контракт и никуда не денется, а вот оставшихся хотя бы частично можно будет демобилизовать. Вероятно, по каким-то критериям, в первую очередь, тех, кто постарше, у кого много детей и т.д. И при условии, что не будет эскалации конфликта с военным положением и всеобщей мобилизацией, конечно.

Общие выводы и советы
На мой взгляд, ничего не указывает на то, что в России готовится вторая волна мобилизации осенью, как пишут некоторые медийные источники. Наоборот, принимаются меры, чтобы этой волны избежать. В реальности, мобилизация уже идет, только совсем другими методами, не теми, что были задействованы прошлой осенью.

При этом, напомню, что у меня нет инсайдов и я не принимаю решения, за чужую глупость я не в ответе. Может быть, я не прав и российское руководство решит зачем-то еще раз прыгнуть на мобилизационные грабли, но вроде я постарался обосновать, почему этого не должны сделать.

В реальности же, если в МО РФ видят, что поток контрактников недостаточен для закрытия потребностей в живой силе, я думаю, что будет задействован механизм настоящей скрытой мобилизации – рассылка повесток ежемесячно без широкой огласки и без резких «волн», чтобы не напрягать армейский тыл и максимально скрывать это от общества.

То есть, если лично вы опасаетесь быть мобилизованным, я бы дал совет не ждать повторения событий прошлого года, а внимательно следить за новостями. Конечно же, не федеральными, а местными, особенно, неформальными СМИ типа групп в Телеграме и ВК, где выкладывают локальные новости. Слушать, что говорит окружение, коллеги и даже читать оппозиционные СМИ, они точно напишут. И обращать внимание не на сам факт рассылки повесток, их постоянно рассылают для сверки данных, а вот на случаи, когда человек пошел, а ему там мобилизационное предписание выписали. Вот когда такое заметите – вот тогда собирайтесь на дачу или в Верхний Ларс, куда запланировано.

В целом же, некая масштабная мобилизация даже в виде волн – возможна, но для нее должны сложиться объективные обстоятельства непосредственно на фронте. И ясны они будут ближе к лету 2024-го года.

Мой канал в ТГ, в нем выходит намного больше разных постов, никаких репостов, исключительно мои авторские тексты: https://t.me/artjockey


#[моё] #Политика #Спецоперация #Война на Украине #Министерство обороны #Мобилизация #Война #Частичная мобилизация #Россия #Призыв #Длиннопост"""

post_request_username = "alex4321"
post_request_character = "you are a programmer, 29 y.o. male, Russian citizen"
post_request_goals = "Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions."

In [ ]:
context_task_creation_response = requests.post(
    "http://127.0.0.1:8080/context",
    json={
        "post": {
            "text": post_text,
            "time": "28 September 2023, 11:28:00",
        },
        "user": {
            "name": post_request_username,
            "character": post_request_character,
            "goals": post_request_goals,
        }
    }
)
assert context_task_creation_response.status_code == 200
context_task_creation_response_data = context_task_creation_response.json()
context_task_creation_response_data

{'id': 26, 'status': 'not_started', 'response': ''}

In [ ]:
context_task_status_response = requests.get(
    "http://127.0.0.1:8080/task-status",
    params={
        "task_id": context_task_creation_response_data["id"],
    }
)
assert context_task_status_response.status_code == 200
context_task_status_response_data = context_task_status_response.json()
context_task_status_response_data

{'id': 26, 'status': 'not_started', 'response': ''}

In [ ]:
wrong_context_task_status_response = requests.get(
    "http://127.0.0.1:8080/task-status",
    params={
        "task_id": context_task_creation_response_data["id"] + 1000,
    }
)
assert wrong_context_task_status_response.status_code == 404

In [ ]:
tasks_response = requests.get(
    "http://127.0.0.1:8080/tasks-for-execution",
    params={"count": 10}
)
assert tasks_response.status_code == 200
tasks_response_data = tasks_response.json()
assert context_task_creation_response_data["id"] in {task["task_id"] for task in tasks_response_data}
tasks_response_data

[{'task_id': 26,
  'task_type': 'context',
  'task_objects': {'context': {'post': {'text': 'Будет ли мобилизация осенью?\u2060\u2060\nСамый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат. Хочу его разобрать и озвучить свои соображения на тему, будет ли мобилизация, если да – то когда.\n\nМой канал в ТГ: https://t.me/artjockey\n\nНапомню старым и новым читателям, что у меня нет никаких инсайдов, я просто озвучу свое мнение и постараюсь его обосновать. С чем можно согласиться или не согласиться.\n\nБудет ли мобилизация осенью? Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост\nПервая волна мобилизации\nОна началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать. То есть, формально, новую волну власть может не объявлять, а просто, опираясь на 

In [ ]:
context_task_status_response = requests.get(
    "http://127.0.0.1:8080/task-status",
    params={
        "task_id": context_task_creation_response_data["id"],
    }
)
assert context_task_status_response.status_code == 200
context_task_status_response_data = context_task_status_response.json()
context_task_status_response_data

{'id': 26, 'status': 'in_progress', 'response': ''}

In [ ]:
_parse_task(tasks_response_data[0])

TaskForExecution(task_id=26, task_type='context', task_objects={'context': ContextRequestWithTime(post=Post(text='Будет ли мобилизация осенью?\u2060\u2060\nСамый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат. Хочу его разобрать и озвучить свои соображения на тему, будет ли мобилизация, если да – то когда.\n\nМой канал в ТГ: https://t.me/artjockey\n\nНапомню старым и новым читателям, что у меня нет никаких инсайдов, я просто озвучу свое мнение и постараюсь его обосновать. С чем можно согласиться или не согласиться.\n\nБудет ли мобилизация осенью? Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост\nПервая волна мобилизации\nОна началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать. То есть, формально, новую волну власть может не объявлять, 

In [ ]:
tasks_parsed = _parse_tasks(tasks_response_data)
tasks_parsed

[TaskForExecution(task_id=26, task_type='context', task_objects={'context': ContextRequestWithTime(post=Post(text='Будет ли мобилизация осенью?\u2060\u2060\nСамый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат. Хочу его разобрать и озвучить свои соображения на тему, будет ли мобилизация, если да – то когда.\n\nМой канал в ТГ: https://t.me/artjockey\n\nНапомню старым и новым читателям, что у меня нет никаких инсайдов, я просто озвучу свое мнение и постараюсь его обосновать. С чем можно согласиться или не согласиться.\n\nБудет ли мобилизация осенью? Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост\nПервая волна мобилизации\nОна началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать. То есть, формально, новую волну власть может не объявлять,

In [ ]:
task_response = await _process_tasks(tasks_parsed)
task_response

[TaskResponse(id=26, status='finished', response='[tags] Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост [/tags]\n\n- Вопрос о возможной мобилизации осенью вызывает множество слухов и спекуляций:\n"Будет ли мобилизация осенью? Самый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат."\n\n- Первая волна мобилизации началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально, указ о мобилизации продолжает юридически действовать:\n"Первая волна мобилизации... с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать."\n\n- Мобилизация была необходима, чтобы поддержать паритет с ВСУ по численности:\n"Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и продолжить войну в условно равном соотношении сил."\n\n- В

In [ ]:
requests.post(
    "http://127.0.0.1:8080/task-status-set",
    json=[
        {"id": task.id, "status": task.status, "response": task.response}
        for task in task_response
    ]
)

<Response [200]>

In [ ]:
context_task_status_response = requests.get(
    "http://127.0.0.1:8080/task-status",
    params={
        "task_id": context_task_creation_response_data["id"],
    }
)
assert context_task_status_response.status_code == 200
context_task_status_response_data = context_task_status_response.json()
context_task_status_response_data

{'id': 26,
 'status': 'finished',
 'response': '[tags] Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост [/tags]\n\n- Вопрос о возможной мобилизации осенью вызывает множество слухов и спекуляций:\n"Будет ли мобилизация осенью? Самый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат."\n\n- Первая волна мобилизации началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально, указ о мобилизации продолжает юридически действовать:\n"Первая волна мобилизации... с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать."\n\n- Мобилизация была необходима, чтобы поддержать паритет с ВСУ по численности:\n"Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и продолжить войну в условно равном соотношении сил."\n\n- Вме

In [ ]:
comment_task_creation_response = requests.post(
    "http://127.0.0.1:8080/comment",
    json={
        "context": context_task_status_response_data["response"],
        "history": [
            {
                "author": "SibirTetushka",
                "time": "22 August 2023, 12:57:00",
                "content": "То есть, я так поняла, эта фиг... СВО растянется ещё года на три, а может, на пять? Конца края этому не видать?"
            },
            {
                "author": "JohnWestDee",
                "time": "22 August 2023, 13:18:00",
                "content": "Автор очень правильно заметил - мы можем находиться в 1914м или 1938м и не знать, что это начало Третьей мировой (тьфу-тьфу, не дай Бог!)"
            },
            {
                "author": "Sintyro",
                "time": "22 August 2023, 14:19:00",
                "content": "А можем еще где то в 1916-1917... И тоже не знать."
            },
            {
                "author": "JohnWestDee",
                "time": "22 August 2023, 14:31:00",
                "content": "Вот честно, тоже не дай Бог. Когда смотришь, как мы подходим к смене власти (Революция и девяностые) - очень разрушительные процессы."
            },
        ],
        "user": {
            "name": "alex4321",
            "character": "you are a programmer, 29 y.o. male",
            "goals": "Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions.",
            "style_example": "\n\n".join([
                "В вакууме, да :Yoba:. Оба же тут существовали всё время или имели таки достигнутое соглашение, чтобы было от чего отталкиваться как опорной точки :Yoba:",
                "Не особо-то может. Не привлекая население в виде не 1% принудительного мармелада и пары процентов добровольного, а в виде процентов 10.\nА то, чтобы привлечь большое количество - неплохо бы, чтобы они понимали, нахуя это им надо. А то так численность военкомов может начать неприемлемо быстро падать, а следом их желание работать.\nА с этим у пропаганды проблема. Вот с чем у них нет проблем, так это с стимуляцией пассивности, но это обратно нужному (для названной вами задачи).\nДа и опять же - ну вот убедишь ты в идее не какого-нибудь Стрелкова и клуб рассерженных долбоёбов, а большое число людей. Что делать, когда (не если, а когда) идея станет неактуальной? Показательной посадкой пары человек дело не закончится же.\n",
                "Точнее не так - смену она не устраивала.\nОна просто выстрелила себе в ногу так, что потом что-то новое приходилось строить не апгрейдом предыдущей системы, а из кусков её трупа.",
            ]),
            "style_description": "- Non-formal style, using mainly Russian language " + \
                "(my English is a bit screwed up)" + \
                "\n- Brief. Most time.\n" + \
                "- Overuse memes sometimes."
        }
    }
)
assert comment_task_creation_response.status_code == 200
comment_task_creation_response_data = comment_task_creation_response.json()
comment_task_creation_response_data

{'id': 27, 'status': 'not_started', 'response': ''}

In [ ]:
tasks_response = requests.get(
    "http://127.0.0.1:8080/tasks-for-execution",
    params={"count": 10}
)
assert tasks_response.status_code == 200
tasks_response_data = tasks_response.json()
assert comment_task_creation_response_data["id"] in {task["task_id"] for task in tasks_response_data}
tasks_response_data

[{'task_id': 27,
  'task_type': 'comment',
  'task_objects': {'comment': {'context': '[tags] Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост [/tags]\n\n- Вопрос о возможной мобилизации осенью вызывает множество слухов и спекуляций:\n"Будет ли мобилизация осенью? Самый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат."\n\n- Первая волна мобилизации началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально, указ о мобилизации продолжает юридически действовать:\n"Первая волна мобилизации... с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать."\n\n- Мобилизация была необходима, чтобы поддержать паритет с ВСУ по численности:\n"Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и продолжить войну в усл

In [ ]:
tasks_parsed = _parse_tasks(tasks_response_data)
tasks_parsed

[TaskForExecution(task_id=27, task_type='comment', task_objects={'comment': CommentRequestWithTime(context='[tags] Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост [/tags]\n\n- Вопрос о возможной мобилизации осенью вызывает множество слухов и спекуляций:\n"Будет ли мобилизация осенью? Самый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат."\n\n- Первая волна мобилизации началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально, указ о мобилизации продолжает юридически действовать:\n"Первая волна мобилизации... с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать."\n\n- Мобилизация была необходима, чтобы поддержать паритет с ВСУ по численности:\n"Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и 

In [ ]:
task_response = await _process_tasks(tasks_parsed)
task_response

[TaskResponse(id=27, status='finished', response='Согласен с вами, ребята. Как говорится: "Война - это когда все теряют". Мы ведь не хотим повторения 1914 или 1938? Или еще хуже, 1917... Нужно тут и сейчас остановиться и задуматься, чего мы добиваемся этой войной. Не пора ли нам уже выключить "Ультра килл" и начать бороться за мир? :Yoba:')]

In [ ]:
requests.post(
    "http://127.0.0.1:8080/task-status-set",
    json=[
        {"id": task.id, "status": task.status, "response": task.response}
        for task in task_response
    ]
)

<Response [200]>

In [ ]:
comment_task_status_response = requests.get(
    "http://127.0.0.1:8080/task-status",
    params={
        "task_id": comment_task_creation_response_data["id"],
    }
)
assert comment_task_status_response.status_code == 200
comment_task_status_response_data = comment_task_status_response.json()
comment_task_status_response_data

{'id': 27,
 'status': 'finished',
 'response': 'Согласен с вами, ребята. Как говорится: "Война - это когда все теряют". Мы ведь не хотим повторения 1914 или 1938? Или еще хуже, 1917... Нужно тут и сейчас остановиться и задуматься, чего мы добиваемся этой войной. Не пора ли нам уже выключить "Ультра килл" и начать бороться за мир? :Yoba:'}